In [ ]:
from unray import render
import numpy as np
import pylab as pl

In [ ]:
# Some values in notebook below are adjusted to the heart model but the code is generic
filename = "data/heart.npz"
#filename = "data/brain.npz"
#filename = "data/aneurysm.npz"

mesh = np.load(filename)
cells = mesh["cells"]
points = mesh["points"]

In [ ]:
# Coordinates of all vertices in mesh
x = list(points.T)  # x[2] = z coordinate array for all vertices

# Model center 3d vector
center = list(map(lambda x: x.mean(), x))

# Model min/max coordinates
bbox = list(map(lambda x: (x.min(), x.max()), x))

# Coordinates with origo shifted to center of model
xm = list(map(lambda x, mp: x - mp, x, center))

# Distance from model center
xd = np.sqrt(sum(map(lambda x: x**2, xm)))
radius = xd.max()

# Distance from center, normalized to max 1.0
func_dist = xd / radius

# A constant for all vertices
func_const = np.ones(x[0].shape)

# x coordinate
func_x = x[0]

# A wave pattern from the center of the model
freq = 4
func_wave = 2.0 + np.sin((freq * 2 * np.pi / radius) * xd)

In [ ]:
subcells = cells.copy()
subpoints = points.copy()

# Pick cells where first vertex is below center in x,y,z
for i in range(3):
    subcells = subcells[np.where(points[subcells[:,0], i] < center[i])]

# Pick cells where first vertex is within an interval
subcells = subcells[np.where(func_dist[subcells[:,0]] < 0.7)]
subcells = subcells[np.where(func_dist[subcells[:,0]] > 0.6)]

# Drop points beyond highest vertex index in subcells
vmap = np.array(sorted(subcells.flatten()), dtype=subcells.dtype)
vmapinv = -np.ones(vmap.max()+1, dtype=subcells.dtype)
vmapinv[vmap] = np.arange(vmap.shape[0], dtype=subcells.dtype)
subpoints2 = subpoints[vmap,:]
subcells2 = vmapinv[subcells]
print(subpoints.shape, subcells.shape)
print(subpoints2.shape, subcells2.shape)

print((lambda a: (a.min(), a.max(), a.mean()))(func_dist[subcells]))

# Pick one cell
if 0:
    subcells = subcells[i:i+1,:]
    subpoints = points[subcells[0],:]
    subcells[:] = [0,1,2,3]

emission_lut = [1,1,1, 0,1,1, 1,1,1, 1,1,1,
                1,1,1, 1,0,1, 1,1,1, 1,1,1,
                1,1,1, 1,1,0, 1,1,1, 1,1,1]
render(cells=subcells, coordinates=subpoints,
       emission=func_dist, emission_lut=emission_lut,
       method="min")